In [ ]:
from grid.refine import *

df = runs.pandas().loc[lambda df: df.description.fillna("").str.startswith("main/")]
df = pd.concat([df, pd.DataFrame(df.params.values.tolist(), df.index)], 1)
df['n_snapshots'] = df._files.apply(lambda d: len([f for f in d if f.startswith('storage.snapshot')]))
df = (df
    .reset_index()
    .groupby(['boardsize', 'width', 'depth'])
    .apply(lambda g: g[g.n_snapshots == g.n_snapshots.max()].iloc[-1])
    .set_index('run'))

invocations = []
for run in df.index:
    for snapshot in list(storage.snapshots(run)) + [None]:
        invocations.append((run, snapshot))
invocations = np.random.permutation(invocations)

archive = jittens.jobs.compress('.', '.jittens/bulk.tar.gz', ['credentials.json'])
for run, snapshot in invocations:
    jittens.jobs.submit(
        cmd=f"""python -c "from grid.refine import *; evaluate({quote(run)}, {snapshot})" >logs.txt 2>&1""", 
        archive=archive, 
        resources={'cpu': 2, 'memory': 4})
    

In [ ]:
rm -rf .jittens

In [ ]:
files.remove(-51, 'arena.json')

In [ ]:
from grid import plots
plots.plot_compute_frontier(df)

In [ ]:
df = data.load()